# Plot SNNS vs Keras scatterplot

In [14]:
import pandas as pd
from bqplot import (
    LinearScale, Scatter, Lines, Figure, Axis, pyplot as plt
)
from ipywidgets import Output, Layout
from IPython.display import display

In [2]:
d_path='/cluster/gjb_lab/2472402/data/dssp_dict.csv'
seq_dict=dict()
with open(d_path,'r') as f:
    rows=f.read().splitlines()
    while (rows):
        row=rows.pop(0)
        domain, seqID = row.split(',')
        seq_dict[int(seqID)]=domain

In [33]:
def plot_scatter(i):
    keras_path="/cluster/gjb_lab/2472402/results/keras/25Aug/cv%d_scores.csv" % i
    snns_path="/cluster/gjb_lab/2472402/results/snns/25Aug/cv%d_scores.csv" % i
    snns=pd.read_csv(snns_path)
    keras=pd.read_csv(keras_path)
    seqIDs=snns.seqID
    snns=snns.set_index('seqID')
    keras=keras.set_index('seqID')
    keras_path_2="/cluster/gjb_lab/2472402/results/keras/25Aug/cv%d.knet" % i
    snns_path_2="/cluster/gjb_lab/2472402/results/snns/25Aug/cv%d.knet" % i
    dssp=[]
    snns_pred=[]
    keras_pred=[]

    # accumulate the 3 lists above
    with open(keras_path_2) as f:
        rows=f.read().splitlines()
        while (rows):
            row=rows.pop(0)
            if (row.startswith('DSSP')):
                dssp.append(row[11:])
            elif (row.startswith('HMM_pred')):
                keras_pred.append(row[11:])
            else:
                continue

    with open(snns_path_2) as f:
        rows=f.read().splitlines()
        while (rows):
            row=rows.pop(0)
            if (row.startswith('HMM_pred')):
                snns_pred.append(row[11:])
            else:
                continue

    dpred=dict()
    for (j,d,s,k) in zip (seqIDs,dssp,snns_pred,keras_pred):
        dpred[j]={'domain':seq_dict[j],'dssp_str':d,'snns_str':s,'keras_str':k}

    x_data=snns.HMM_acc
    y_data=keras.HMM_acc

    x_sc=LinearScale()
    y_sc=LinearScale()

    ax_x = Axis(label='SNNS HMM acc', scale=x_sc, tick_format='0.1f')
    ax_y = Axis(label='Keras HMM acc', scale=y_sc, orientation='vertical', tick_format='0.1f')

    # create Output widget
    out=Output()
    out.layout={'max_width':'95%'}

    # define function to run when I hover over a mark
    def def_tt(_,event):
        out.clear_output()
        with out:
            seqID=event['data']['name']
            val_x=event['data']['x']
            val_y=event['data']['y']
            domain=dpred[seqID]['domain']
            dssp_str=dpred[seqID]['dssp_str']
            snns_str=dpred[seqID]['snns_str']
            keras_str=dpred[seqID]['keras_str']
            print('seqID:  %s' % seqID)
            print('SCOPe:  %s' % domain)
            print('snns:   %s' % val_x)
            print('keras:  %s' % val_y)
            print('dssp:   %s' % dssp_str)
            print('snns:   %s' % snns_str)
            print('keras:  %s' % keras_str)

    scatter=Scatter(x=x_data,
                    y=y_data,
                    scales={'x': x_sc, 'y': y_sc},
                    names=seqIDs,
                    display_names=False,
                    tooltip=out,
                    opacities=[0.5],
                    hovered_style={'opacity':1.0,'fill':'DarkOrange'},
                    unhovered_style={'opacity':0.2,},
                   )

    scatter.on_hover(def_tt)

    line=Lines(x=[60,100],y=[60,100],
               scales={'x': x_sc, 'y': y_sc},
               colors=['White'],
               stroke_width=0.5,
               line_style='dashed'
              )

    fig=Figure(title='Cross-validation fold %d accuracies' % i,
           marks=[scatter,line],
           axes=[ax_x,ax_y],
           layout={'height':'900px','width':'900px'},
          )
    
    display(fig)

In [37]:
CROSS_VAL_FOLD=2
plot_scatter(CROSS_VAL_FOLD)

Figure(axes=[Axis(label='SNNS HMM acc', scale=LinearScale(), tick_format='0.1f'), Axis(label='Keras HMM acc', …